# **DESCRIPTION**

This project consist in a forecasting of cost for different industrial supplies from a company

# **Project Develop**

## *Install Libraries*

In [13]:
# !pip install pandas
# !pip install numpy
# !pip install openpyxl

## *Import Libraries*

In [14]:
import pandas as pd
import numpy as np
import openpyxl
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## *Import data*

Import data from an Excel file provided by the company

In [15]:
df_purchases = pd.read_excel("COMPRAS.xlsx", index_col=0)

In [16]:
df_purchases.head(10)

,supplier_order_id,order_date,supplier_name,position_supply,supply_id,supply_reference,unit_value,discount,delivery_date,quantity,pending,deliv_date_1,deliv_quant_1,deliv_note_1,deliv_date_2,deliv_quant_2,deliv_note_2,deliv_date_3,deliv_quant_3,deliv_note_3
id,,,,,,,,,,,,,,,,,,,,
5252,64,2013-05-16,OFFICINE SANTAFEDE,1.0,16556,"BWN 6""900RTJ ID142,88mm F44",1020.00,0.0,2013-11-18,50.0,0.0,NaN,28.0,NaN,NaN,22.0,NaN,NaN,0.0,NaN
5253,64,2013-05-16,OFFICINE SANTAFEDE,2.0,16557,FORGING ROUND F44,2560.00,0.0,2013-11-18,25.0,0.0,NaN,25.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5589,1,2013-09-02,Aceros y Equipos S.L.,1.0,16548,"BRE 76,2MM HAST C276",44.50,0.0,2013-09-04,1.6,0.0,NaN,1.6,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5590,2,2013-09-03,UTILES Y MAQUINAS INDUSTRIALES,1.0,15728,"TP1R 0,5mm",420.60,0.0,2013-09-10,3.0,0.0,NaN,3.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5591,2,2013-09-03,UTILES Y MAQUINAS INDUSTRIALES,2.0,16383,VAR 4.5X2X1.2X1000 A710,19.60,0.0,2013-09-10,5.0,0.0,NaN,5.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5592,3,2013-09-18,OFFICINE SANTAFEDE,1.0,15774,"TPI 4-1/16""X4""2500",5200.00,0.0,2013-10-21,1.0,0.0,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5502,4,2013-09-20,ThyssenKrupp Materials Ibérica,3.0,12029,BRE 165 AISI-304L,2.55,0.0,2013-10-08,112.0,0.0,NaN,112.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5503,5,2013-09-20,Aceros y Equipos S.L.,1.0,11819,BRE 55 AISI-321,4.70,0.0,2013-09-24,112.0,0.0,NaN,112.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
5504,6,2013-09-20,"Empresa Santa Lucía, S.A.",1.0,11715,BRE 35 A5,14.60,0.0,2013-09-27,132.0,0.0,NaN,132.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


## *Data Resume*

In [17]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10775 entries, 5252 to 10818
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   supplier_order_id  10775 non-null  int64         
 1   order_date         10775 non-null  datetime64[ns]
 2   supplier_name      10775 non-null  object        
 3   position_supply    10763 non-null  float64       
 4   supply_id          10775 non-null  int64         
 5   supply_reference   10775 non-null  object        
 6   unit_value         10775 non-null  float64       
 7   discount           10775 non-null  float64       
 8   delivery_date      10420 non-null  datetime64[ns]
 9   quantity           10775 non-null  float64       
 10  pending            10775 non-null  float64       
 11  deliv_date_1       574 non-null    float64       
 12  deliv_quant_1      10692 non-null  float64       
 13  deliv_note_1       574 non-null    object        
 14  deliv_da

In [18]:
df_purchases.describe()

,supplier_order_id,order_date,position_supply,supply_id,unit_value,discount,delivery_date,quantity,pending,deliv_date_1,deliv_quant_1,deliv_date_2,deliv_quant_2,deliv_date_3,deliv_quant_3
count,10775.000000,10775,10763.000000,10775.000000,1.077500e+04,10775.000000,10420,1.077500e+04,10775.000000,574.000000,1.069200e+04,248.000000,10692.000000,31.000000,10692.000000
mean,1466.358515,2019-04-04 08:59:46.969837568,8.807582,15884.098840,1.934475e+04,0.103745,2019-02-21 04:31:00.115163136,1.707657e+10,0.663651,45467.355401,1.720913e+10,45354.991935,7.826178,45334.161290,1.836050
min,1.000000,2013-05-16 00:00:00,1.000000,11200.000000,0.000000e+00,0.000000,2012-11-30 00:00:00,0.000000e+00,0.000000,45295.000000,0.000000e+00,45295.000000,0.000000,45294.000000,0.000000
25%,818.500000,2016-10-03 12:00:00,1.000000,13938.500000,4.260000e+00,0.000000,2016-09-05 00:00:00,2.000000e+00,0.000000,45404.000000,1.000000e+00,45310.000000,0.000000,45294.000000,0.000000
50%,1485.000000,2019-01-29 00:00:00,3.000000,15834.000000,1.950000e+01,0.000000,2018-11-12 00:00:00,6.000000e+00,0.000000,45455.000000,4.000000e+00,45334.000000,0.000000,45348.000000,0.000000
75%,2151.500000,2022-01-07 00:00:00,9.000000,17876.500000,1.000000e+02,0.000000,2021-09-07 00:00:00,3.000000e+01,0.000000,45544.000000,2.400000e+01,45370.000000,0.000000,45369.000000,0.000000
max,2903.000000,2024-12-16 00:00:00,105.000000,20060.000000,1.000000e+07,40.000000,2024-12-02 00:00:00,1.840000e+14,1500.000000,45628.000000,1.840000e+14,45635.000000,5000.000000,45373.000000,11700.000000
std,824.794745,NaN,13.565559,2375.611046,2.905807e+05,1.417428,NaN,1.772594e+12,16.886231,84.396032,1.779460e+12,64.945009,87.111424,33.843854,116.367555


## *Data Engineering*

Columns that do not provide data of interest are eliminated.

In [19]:
df_purchases = df_purchases.drop(columns=["supplier_order_id","position_supply","supply_id","discount","pending",
                    "deliv_date_1","deliv_quant_1","deliv_note_1",
                    "deliv_date_2","deliv_quant_2","deliv_note_2",
                    "deliv_date_3","deliv_quant_3","deliv_note_3"])

Change the order of columns in dataframe

In [20]:
new_column_order = ["order_date", "delivery_date", "supplier_name", "supply_reference","unit_value","quantity"]
df_purchases = df_purchases[new_column_order]

Calculation of the relative change in the unit price of a product compared to previous purchases

In [ ]:
df_purchases = df_purchases.sort_values(by=['supply_reference', 'order_date'])

# Calculate the previous unit price for each product
df_purchases['previous_unit_value'] = df_purchases.groupby('supply_reference')['unit_value'].shift(1)

# Calculate the rate of change in the unit price
df_purchases['price_change_rate'] = ((df_purchases['unit_value'] - df_purchases['previous_unit_value']) / df_purchases['previous_unit_value']) * 100

# Fill the NaN values (which appear for the first purchase of each product) with 0 or an appropriate value
df_purchases['price_change_rate'] = df_purchases['price_change_rate'].fillna(0)

Verify if infinite or NaN values in new colum

In [ ]:
num_infinite_values = np.isinf(df_purchases['price_change_rate']).sum()
num_nan_values = df_purchases['price_change_rate'].isnull().sum()

print(f"Infinites values: {num_infinite_values}; NaN values: {num_nan_values}")

Replacing infinite values

In [ ]:
df_purchases['price_change_rate'].replace([np.inf, -np.inf], np.nan, inplace=True)
mean_value = df_purchases['price_change_rate'].mean()
df_purchases['price_change_rate'].fillna(mean_value, inplace=True)

In [ ]:
num_infinite_values = np.isinf(df_purchases['price_change_rate']).sum()
num_nan_values = df_purchases['price_change_rate'].isnull().sum()

print(f"Infinites values: {num_infinite_values}; NaN values: {num_nan_values}")

Coding cathegorical variables. Using Target Encoding to establis to each category the mean of target variable

In [24]:
supplier_avg_cost = df_purchases.groupby("supplier_name")["unit_value"].mean()
supply_ref_avg_cost = df_purchases.groupby("supply_reference")["unit_value"].mean()

df_purchases["supplier_encoded"] = df_purchases["supplier_name"].map(supplier_avg_cost)
df_purchases["supply_ref_encoded"] = df_purchases["supply_reference"].map(supply_ref_avg_cost)

,order_date,delivery_date,supplier_name,supply_reference,unit_value,quantity,total_cost,supplier_encoded,supply_ref_encoded
id,,,,,,,,,
5252,2013-05-16,2013-11-18,OFFICINE SANTAFEDE,"BWN 6""900RTJ ID142,88mm F44",1020.0,50.0,51000.0,2.239374e+05,111500.000000
5253,2013-05-16,2013-11-18,OFFICINE SANTAFEDE,FORGING ROUND F44,2560.0,25.0,64000.0,2.239374e+05,64000.000000
5589,2013-09-02,2013-09-04,Aceros y Equipos S.L.,"BRE 76,2MM HAST C276",44.5,1.6,71.2,8.413519e+12,71.200000
5590,2013-09-03,2013-09-10,UTILES Y MAQUINAS INDUSTRIALES,"TP1R 0,5mm",420.6,3.0,1261.8,3.697784e+03,1394.140000
5591,2013-09-03,2013-09-10,UTILES Y MAQUINAS INDUSTRIALES,VAR 4.5X2X1.2X1000 A710,19.6,5.0,98.0,3.697784e+03,33062.485714


Creation of new categories for time series

In [25]:
df_purchases["lead_time"] = (df_purchases["delivery_date"] - df_purchases["order_date"]).dt.days  # Delivery tim in days
df_purchases["month"] = df_purchases["order_date"].dt.month  # Month of order
df_purchases["year"] = df_purchases["order_date"].dt.year # Year of order

,order_date,delivery_date,supplier_name,supply_reference,unit_value,quantity,total_cost,supplier_encoded,supply_ref_encoded,lead_time,month,year
id,,,,,,,,,,,,
5252,2013-05-16,2013-11-18,OFFICINE SANTAFEDE,"BWN 6""900RTJ ID142,88mm F44",1020.0,50.0,51000.0,2.239374e+05,111500.000000,186.0,5,2013
5253,2013-05-16,2013-11-18,OFFICINE SANTAFEDE,FORGING ROUND F44,2560.0,25.0,64000.0,2.239374e+05,64000.000000,186.0,5,2013
5589,2013-09-02,2013-09-04,Aceros y Equipos S.L.,"BRE 76,2MM HAST C276",44.5,1.6,71.2,8.413519e+12,71.200000,2.0,9,2013
5590,2013-09-03,2013-09-10,UTILES Y MAQUINAS INDUSTRIALES,"TP1R 0,5mm",420.6,3.0,1261.8,3.697784e+03,1394.140000,7.0,9,2013
5591,2013-09-03,2013-09-10,UTILES Y MAQUINAS INDUSTRIALES,VAR 4.5X2X1.2X1000 A710,19.6,5.0,98.0,3.697784e+03,33062.485714,7.0,9,2013


Standarization of numeric columns

In [ ]:
scaler = StandardScaler()

df_purchases['quantity'] = scaler.fit_transform(df_purchases[['quantity']])
df_purchases['unit_value'] = scaler.fit_transform(df_purchases[['unit_value']])
df_purchases['lead_time'] = scaler.fit_transform(df_purchases[['lead_time']])

df_purchases.head(5)

Get X and Y variables droping those columns without interesting data

In [ ]:
X = df_purchases[['quantity', 'price_change_rate', 'supplier_encoded', 'supply_ref_encoded', 'lead_time', 'month', 'year']]
y = df_purchases['unit_value']

Divide data in train and test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Train and prediction each model looking for the best

In [ ]:
# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_lin = linear_model.predict(X_test)

# Decission Tree
tree_model = DecisionTreeRegressor(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)

#Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Gradient Boosting
gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(X_train, y_train)
y_pred_gb = gb_model.predict(X_test)

# Light Gradient Boostin
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)


models = {
    "Linear Regression": linear_model,
    "Decision Tree": tree_model,
    "Random Forest": rf_model,
    "Gradient Boosting": gb_model,
    "LightGBM": lgb_model
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{name}: MAE = {mae:.2f}, RMSE = {rmse:.2f}, R² = {r2:.2f}")

Cross validation to check better results of train/test data

In [ ]:
models = {
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'LightGBM': lgb.LGBMRegressor()
}

# Using cross-validation to calculate R²
results = {}
for model_name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    results[model_name] = scores

# Showing results
for model_name, scores in results.items():
    print(f"{model_name}:")
    print(f" R²: {scores}")
    print(f" Mean/STD R²: {np.mean(scores):.4f} +/- {np.std(scores):.4f}\n")